old name: polygons_data_preprocessing_marks.ipynb

In [19]:
import os
from glob import glob
from pathlib import Path
import pickle

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from utils.plotting import plot_trajectory
from utils.processing import (
    normalize_angles_2pi,
    open_file,
    cut_jumps
    )
from utils.processing import add_filtered_V


import seaborn as sns
from scipy import stats
#from scipy.ndimage import gaussian_filter1d
# from utils.processing import (
#     normalize_angles_2pi,
#     open_file,
#     cut_jumps,
#     save_preprocessed_data,
#     open_preprocessed_data,
#     )

## basic processing without resampling

#####  parameters to change

In [2]:
wall_perc = 0.1

##### code

In [3]:
DATA_DIR = 'dataset'
PREPROC_DATA_DIR = 'downsamp_preprocessed_dataset'

# 1. open files
dataset_names = sorted(glob(os.path.join(DATA_DIR, '*.txt')))
datasets = {}
for nam in dataset_names:
    k = Path(nam).stem
    v = open_file(nam, show=False)
    datasets[k] = v

# 2. cut_jumps
tr11 = datasets['Trace_1.1']  # TODO: params to config
tr12 = datasets['Trace_1.2']  
tr2 = datasets['Trace_2']
tr5 = datasets['Trace_5']

tr4 = cut_jumps(datasets['Trace_4'], xmax=40.4, xmin=-38.1, ymax=28.3, ymin=-47.6,show=False)
tr3 = cut_jumps(datasets['Trace_3'], xmax=46.8, xmin=-48, ymax=46, ymin=-47,show=False)
tr6 = cut_jumps(datasets['Trace_6'], xmax=45, xmin=-45, ymax=45, ymin=-42.2,show=False)
tr7 = cut_jumps(datasets['Trace_7'], xmax=35, xmin=-38.7, ymax=40, ymin=-32,show=False)
tr91 = cut_jumps(datasets['Trace_9.1'], xmax=42.5, xmin=-43.7, ymax=42.5, ymin=-42.7,show=False)
tr92 = cut_jumps(datasets['Trace_9.2'], xmax=36.3, xmin=-46.5, ymax=45.4, ymin=-47.3,show=False)

dfs = [tr11, tr12, tr2, tr3, tr4, tr5, tr6, tr7, tr91, tr92]
dfs_names = ['tr11', 'tr12', 'tr2','tr3','tr4','tr5','tr6','tr7', 'tr91','tr92']

dfs_dict = {}
for k, val in zip(dfs_names, dfs):
    dfs_dict[k] = val

# 3. rescale x and y coords to fit area size 
# может добавить функцию rescale_coord в processing.py? TODO
def rescale_coord(data, new_min, new_max):
    min_old_x, max_old_x = np.min(data), np.max(data)
    data_rescaled = ((data - min_old_x) / (max_old_x - min_old_x)) * (new_max - new_min) + new_min
    return data_rescaled

AREA_SIZE = [-40, 40]
rescaled_dfs_dict = {}
for k, df in dfs_dict.items():
    df = df.dropna()
    rescaled_x = rescale_coord(df['x'].to_numpy(), AREA_SIZE[0], AREA_SIZE[1])
    rescaled_y = rescale_coord(df['y'].to_numpy(), AREA_SIZE[0], AREA_SIZE[1])
    df.loc[:, 'x'] = rescaled_x
    df.loc[:, 'y'] = rescaled_y

    rescaled_dfs_dict[k] = df

# 4. divide trajectories into sub-trajectories to remove inconsistent dt and Nans
def divide_traj_by_nans(df, k, dt_threshold, traj_drop_threshold=5):
    # 1. Drop Nans
    df = df.dropna()
    # 2. Find if any time jumps
    dt_times = df['time'].diff().to_numpy()
    dt_times[0] = 0.
    div_idx = np.where(dt_times > dt_threshold)[0]
    div_idx = np.asarray(div_idx)

    # Divide by time jumps
    if len(div_idx) > 0:
        div_idx = np.concatenate([np.array([0]),
                                  div_idx,
                                  np.array([len(df)])])
        sub_trajectories = []
        start_i = div_idx[0]
        for end_i in div_idx[1:]:
            if end_i - start_i > traj_drop_threshold:
                sub_df = df[start_i:end_i]
                sub_trajectories.append(sub_df)
            start_i = end_i + 1

        return k, sub_trajectories
    else:
        return k, [df]
    
DROP_THRESHOLD = 5
dt_thresholds = {'tr11': 0.035, 'tr12': 0.035, 'tr2': 0.035, 'tr3': 0.135,  # TODO: to configs
                 'tr4': 0.035,  'tr5': 0.035, 'tr6': 0.035, 'tr7': 0.035,
                 'tr91': 0.035, 'tr92': 0.035}

subtraj_dfs_dict = {}
for k, df in rescaled_dfs_dict.items():
    k, df_subtraj= divide_traj_by_nans(df, k, dt_threshold=dt_thresholds[k],
                                       traj_drop_threshold=DROP_THRESHOLD)
    subtraj_dfs_dict[k] = df_subtraj  # list of dfs


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8324\1635007879.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'x'] = rescaled_x
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8324\1635007879.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'y'] = rescaled_y
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8324\1635007879.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

NEW_DT - 1/ new sampling rate 



In [4]:
# 5. resample to required dt

# 6.1 add filtered v

for k, df in subtraj_dfs_dict.items():
    sub_list = []
    i = 0
    for ldf in df:
        ldf = ldf.reset_index(drop=True)
        ldf = add_filtered_V(ldf)
        sub_list.append(ldf)
    subtraj_dfs_dict[k] = sub_list



In [5]:
# 6.2 add angles and steps

def add_angles(df):
    dt = df['time'].diff()
    vvx = df['x'].diff() / dt 
    vvy = df['y'].diff() / dt 
    dt[0], vvx[0], vvy[0] = 0., 0., 0.

    Xx = np.arctan2(vvx, vvy)
    Nres = normalize_angles_2pi(Xx)
    ang = np.rad2deg(Nres) % 360
    df['angles'] = ang
    return df

# def add_delta_angles(df):
#     dangles = df['angles'].diff()
#     dangles[0] = 0.
#     dangles_rad = np.deg2rad(dangles)
#     normed_dangels_rad = normalize_angles_2pi(dangles_rad)
#     df['delta_angle'] = np.rad2deg(normed_dangels_rad) % 360
#     return df


def add_step_length(df):
    dx = df['x'].diff()
    dy = df['y'].diff()
    dx[0], dy[0] = 0., 0.,
    step_lens = (dx**2 + dy**2)**0.5
    df['step_length'] = step_lens
    return df

In [6]:
dfs_dict_with_angles = {}
for k, df in subtraj_dfs_dict.items():
    subtraj_list = []
    for sub_traj_df in df:
        df_step = add_step_length(sub_traj_df)
        df_ang = add_angles(df_step)
        #df_dt_ang = add_delta_angles(df_ang)
        subtraj_list.append(df_ang)
    dfs_dict_with_angles[k] = subtraj_list

In [7]:
# dfs_dict_with_angles = {}
# for k, df in subtraj_dfs_dict.items():
#     subtraj_list = []
#     for sub_traj_df in df:
#         df_step = add_step_length(sub_traj_df)
#         subtraj_list.append(df_step)
#     dfs_dict_with_angles[k] = subtraj_list

# 7. sort by areas of open field
#долго выполняется если рисовать все графики


def sort_walls_area(tr, border_tr, wall_percent=0.1): 
    ylen = border_tr['ymax'] - border_tr['ymin'] 
    xlen = border_tr['xmax'] - border_tr['xmin'] 
    xl = border_tr['xmin'] + xlen * wall_percent
    xr = border_tr['xmax'] - xlen * wall_percent
    yd = border_tr['ymin'] + ylen * wall_percent
    yu = border_tr['ymax'] - ylen * wall_percent

    tr['near_wall'] = 0.
    for i in range(len(tr)):     
        if tr['x'].iloc[i] < xl:
            if tr['y'].iloc[i] > yu:
                tr['near_wall'].iloc[i] = 40
            elif tr['y'].iloc[i] < yd:
                tr['near_wall'].iloc[i] = 30
            else:
                tr['near_wall'].iloc[i] = 4
        elif tr['x'].iloc[i] > xr:
            if tr['y'].iloc[i] > yu:
                tr['near_wall'].iloc[i] = 10
            elif tr['y'].iloc[i] < yd:
                tr['near_wall'].iloc[i] = 20
            else:
                tr['near_wall'].iloc[i] = 2
        elif tr['y'].iloc[i] > yu:
            tr['near_wall'].iloc[i] = 1
        elif tr['y'].iloc[i] < yd:
            tr['near_wall'].iloc[i] = 3
    return tr

def plot_wall_sorted_samples(df, plot_title):
    all_areas = [40, 30, 20, 10, 4, 3, 2, 1, 0]
    for a in all_areas:
        x = df.loc[df['near_wall'] == a]['x']
        y = df.loc[df['near_wall'] == a]['y']
        plt.scatter(x, y, s=1.5, label=str(a))
    plt.gca().set_aspect('equal')
    plt.title(plot_title)
    plt.legend(loc='center')
    plt.show()


dfs_with_walls = {}
bord40 =  {'xmin': -40,'xmax': 40,'ymin': -40,'ymax': 40}   

for k, df in dfs_dict_with_angles.items():
    subtraj_list = []
    for sub_df in df:
        sub_df = sort_walls_area(sub_df, bord40, wall_perc)
        #plot_wall_sorted_samples(sub_df, k)
        subtraj_list.append(sub_df)
    dfs_with_walls[k] = subtraj_list







In [9]:
#dfs_with_walls

In [9]:
# #checked that sampling rate stability in subtrajectories
# for k, df in dfs_with_walls.items():
#     print(k)
#     for ldf in df:
#         s_r = (ldf['time'].max() - ldf['time'].min())/ len(ldf)
#         print(s_r)

tr11
0.03336604017653325
tr12
0.03336518498802236
0.03336519800681878
0.03304854368932035
0.033362012290981846
tr2
0.03336603694213928
tr3
0.13117241379310346
0.11866666666666642
0.13021951219512193
0.12558823529411794
0.12136363636363709
0.12994736842105306
0.13068749999999993
0.12642105263157974
0.12903333333333364
0.12714285714285656
0.1304318181818183
0.11674999999999969
0.12851851851851848
0.11675000000000324
0.11866666666666485
0.1303720930232557
0.1302857142857138
0.11866666666666485
0.1255882352941171
0.1304888888888892
0.11442857142856967
0.1314776119402989
0.12942424242424116
0.13043181818181762
0.1297777777777785
0.12769565217391357
0.13175641025641008
0.11674999999999613
0.1268000000000029
0.13123333333333373
0.1302857142857138
0.132182692307693
0.12233333333333007
0.13056521739130372
0.13273076923076946
0.131349206349206
0.12564705882352747
0.11674999999999613
0.12679999999999722
0.13238211382113835
0.12983783783783712
0.12679999999999722
0.13043181818181893
0.132496350364

In [10]:
# 7.5 differentiate movements and stops, check percent of movement

#add sum of steps for the last 0.5 seconds to each row (if applicable)

dfs_step_sum = {}

#window_size = round(0.5/0.034)

for k, df in dfs_with_walls.items():
    sub_list = []
    for ldf in df:
        ldf = ldf.iloc[1:]
        #first row of ldf does not have step length
        s_r = (ldf['time'].max() - ldf['time'].min())/ len(ldf)
        window_size = round(0.5/s_r)
        if len(ldf) >= window_size:
            ldf['step_sum_0.5'] = ldf['step_length'].rolling(window_size).sum()
            ldf['step_sum_0.5']=ldf['step_sum_0.5'].fillna(-100)
            sub_list.append(ldf)
        dfs_step_sum[k] = sub_list
        
#merge dfs in dictionary

merg_resampled_dfs_dict = {}

for k, df in dfs_step_sum.items():
    df = [df0.iloc[1:].reset_index() for df0 in df]
    merg_resampled_dfs_dict[k] = pd.concat(df)
    merg_resampled_dfs_dict[k] = merg_resampled_dfs_dict[k].reset_index(drop=True)



C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8324\1996961697.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['step_sum_0.5'] = ldf['step_length'].rolling(window_size).sum()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8324\1996961697.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf['step_sum_0.5']=ldf['step_sum_0.5'].fillna(-100)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8324\1996961697.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [25]:
# def movementOrNot(step_sum, max_step_of_stop):
#     if step_sum == -100:
#         out = 'NotDefined'
#     elif step_sum > max_step_of_stop:
#         out = 'Move'
#     elif step_sum <= max_step_of_stop:
#         out = 'Stop'
#     else:
#         out = 'error'
#     return out

# def addMovementColumn(dict_df, threshold):
#     new_dict = {}
#     for k, df in dict_df.items():
#         func = np.vectorize(movementOrNot)
#         df["movement"] = func(df['step_sum_0.5'], threshold)
#         new_dict[k] = df
#     return new_dict
    

In [12]:
# # Save distributions

#dfs_stop_save = addMovementColumn(merg_resampled_dfs_dict, 2.5)

with open('for_polygons_basic.pkl', 'wb') as fp:    
    pickle.dump(merg_resampled_dfs_dict, fp)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(dfs_stop['tr11'][0].loc[800:1100])


## resampled to 0.1

#####  parameters to change

In [20]:
wall_perc = 0.1
#parameter to change! time lag between neighboring points 
NEW_DT = 0.1

##### code

In [21]:
DATA_DIR = 'dataset'
PREPROC_DATA_DIR = 'downsamp_preprocessed_dataset'

# 1. open files
dataset_names = sorted(glob(os.path.join(DATA_DIR, '*.txt')))
datasets = {}
for nam in dataset_names:
    k = Path(nam).stem
    v = open_file(nam, show=False)
    datasets[k] = v

# 2. cut_jumps
tr11 = datasets['Trace_1.1']  # TODO: params to config
tr12 = datasets['Trace_1.2']  
tr2 = datasets['Trace_2']
tr5 = datasets['Trace_5']

tr4 = cut_jumps(datasets['Trace_4'], xmax=40.4, xmin=-38.1, ymax=28.3, ymin=-47.6,show=False)
tr3 = cut_jumps(datasets['Trace_3'], xmax=46.8, xmin=-48, ymax=46, ymin=-47,show=False)
tr6 = cut_jumps(datasets['Trace_6'], xmax=45, xmin=-45, ymax=45, ymin=-42.2,show=False)
tr7 = cut_jumps(datasets['Trace_7'], xmax=35, xmin=-38.7, ymax=40, ymin=-32,show=False)
tr91 = cut_jumps(datasets['Trace_9.1'], xmax=42.5, xmin=-43.7, ymax=42.5, ymin=-42.7,show=False)
tr92 = cut_jumps(datasets['Trace_9.2'], xmax=36.3, xmin=-46.5, ymax=45.4, ymin=-47.3,show=False)

dfs = [tr11, tr12, tr2, tr3, tr4, tr5, tr6, tr7, tr91, tr92]
dfs_names = ['tr11', 'tr12', 'tr2','tr3','tr4','tr5','tr6','tr7', 'tr91','tr92']

dfs_dict = {}
for k, val in zip(dfs_names, dfs):
    dfs_dict[k] = val

# 3. rescale x and y coords to fit area size 
# может добавить функцию rescale_coord в processing.py? TODO
def rescale_coord(data, new_min, new_max):
    min_old_x, max_old_x = np.min(data), np.max(data)
    data_rescaled = ((data - min_old_x) / (max_old_x - min_old_x)) * (new_max - new_min) + new_min
    return data_rescaled

AREA_SIZE = [-40, 40]
rescaled_dfs_dict = {}
for k, df in dfs_dict.items():
    df = df.dropna()
    rescaled_x = rescale_coord(df['x'].to_numpy(), AREA_SIZE[0], AREA_SIZE[1])
    rescaled_y = rescale_coord(df['y'].to_numpy(), AREA_SIZE[0], AREA_SIZE[1])
    df.loc[:, 'x'] = rescaled_x
    df.loc[:, 'y'] = rescaled_y

    rescaled_dfs_dict[k] = df

# 4. divide trajectories into sub-trajectories to remove inconsistent dt and Nans
def divide_traj_by_nans(df, k, dt_threshold, traj_drop_threshold=5):
    # 1. Drop Nans
    df = df.dropna()
    # 2. Find if any time jumps
    dt_times = df['time'].diff().to_numpy()
    dt_times[0] = 0.
    div_idx = np.where(dt_times > dt_threshold)[0]
    div_idx = np.asarray(div_idx)

    # Divide by time jumps
    if len(div_idx) > 0:
        div_idx = np.concatenate([np.array([0]),
                                  div_idx,
                                  np.array([len(df)])])
        sub_trajectories = []
        start_i = div_idx[0]
        for end_i in div_idx[1:]:
            if end_i - start_i > traj_drop_threshold:
                sub_df = df[start_i:end_i]
                sub_trajectories.append(sub_df)
            start_i = end_i + 1

        return k, sub_trajectories
    else:
        return k, [df]
    
DROP_THRESHOLD = 5
dt_thresholds = {'tr11': 0.035, 'tr12': 0.035, 'tr2': 0.035, 'tr3': 0.135,  # TODO: to configs
                 'tr4': 0.035,  'tr5': 0.035, 'tr6': 0.035, 'tr7': 0.035,
                 'tr91': 0.035, 'tr92': 0.035}

subtraj_dfs_dict = {}
for k, df in rescaled_dfs_dict.items():
    k, df_subtraj= divide_traj_by_nans(df, k, dt_threshold=dt_thresholds[k],
                                       traj_drop_threshold=DROP_THRESHOLD)
    subtraj_dfs_dict[k] = df_subtraj  # list of dfs


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13432\1635007879.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'x'] = rescaled_x
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13432\1635007879.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'y'] = rescaled_y
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13432\1635007879.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

NEW_DT - 1/ new sampling rate 



In [22]:
# 5. resample to required dt


def resample_data_to_larger_timestep(df, original_dt, new_dt):
    from scipy.signal import resample

    num_original_points = df.shape[0]
    duration = num_original_points * original_dt
    num_new_points = int(duration / new_dt)
    
    resampled_dfnp = resample(df.to_numpy()[:, 1:], num_new_points, axis=0)
    new_time = np.linspace(df['time'].iloc[0], df['time'].iloc[-1], num_new_points)
    #added 3 down change
    #print(resampled_dfnp[:, 0])
    #print(resampled_dfnp)
    if len(resampled_dfnp) > 1:
        resampled_df = pd.DataFrame({'time': new_time,
                                     'x': resampled_dfnp[:, 0], 
                                     'y': resampled_dfnp[:, 1]})
    #added 2 down change
    else:
        resampled_df = pd.DataFrame(columns=['time','x','y'])
    return resampled_df

# NOTE: We ignore that there are TWO most frequent dt-s: 0.033 and 0.034
ORIGINAL_DT = {'tr11': 0.034, 'tr12': 0.034, 'tr2': 0.034, 'tr3': 0.134,  # TODO: to configs
               'tr4': 0.034, 'tr5': 0.034, 
               'tr6': 0.034, 'tr7': 0.034, 'tr91': 0.034, 'tr92': 0.034}


resampled_subtraj_dfs_dict = {}
for k, df in subtraj_dfs_dict.items():
    subtraj_list = []
    for sub_traj_df in df:
        resampled_sub_traj_df = resample_data_to_larger_timestep(sub_traj_df,
                                                                    ORIGINAL_DT[k],
                                                                    NEW_DT)
        #changed
        if len(resampled_sub_traj_df) > 1:
            subtraj_list.append(resampled_sub_traj_df)
    resampled_subtraj_dfs_dict[k] = subtraj_list






In [27]:
dfs_dict_with_angles.keys()

dict_keys(['tr11', 'tr12', 'tr2', 'tr3', 'tr4', 'tr5', 'tr6', 'tr7', 'tr91', 'tr92'])

In [24]:
# 6 add angles, delta angles, steps

def add_angles(df):
    dt = df['time'].diff()
    vvx = df['x'].diff() / dt 
    vvy = df['y'].diff() / dt 
    dt[0], vvx[0], vvy[0] = 0., 0., 0.

    Xx = np.arctan2(vvx, vvy)
    Nres = normalize_angles_2pi(Xx)
    ang = np.rad2deg(Nres) % 360
    df['angles'] = ang
    return df

def add_delta_angles(df):
    dangles = df['angles'].diff()
    dangles[0] = 0.
    dangles_rad = np.deg2rad(dangles)
    normed_dangels_rad = normalize_angles_2pi(dangles_rad)
    df['delta_angle'] = np.rad2deg(normed_dangels_rad) % 360
    return df


def add_step_length(df):
    dx = df['x'].diff()
    dy = df['y'].diff()
    dx[0], dy[0] = 0., 0.,
    step_lens = (dx**2 + dy**2)**0.5
    df['step_length'] = step_lens
    return df

dfs_dict_with_angles = {}
for k, df in resampled_subtraj_dfs_dict.items():
    subtraj_list = []
    for sub_traj_df in df:
        df_ang = add_angles(sub_traj_df)
        df_dt_ang = add_delta_angles(df_ang)
        df_step = add_step_length(df_dt_ang)
        subtraj_list.append(df_step)
    dfs_dict_with_angles[k] = subtraj_list

In [12]:
# for k, df in dfs_dict_with_angles.items():
#     for sub_traj_df in df:
#         s_r = (sub_traj_df['time'].max() - sub_traj_df['time'].min())/ len(sub_traj_df)
#         print(s_r)

0.09813607116880212
0.09813648225469729
0.0981394960143996
0.09725714285714275
0.09816400336417155
0.09813716888813903
0.09880519480519481
0.08899999999999982
0.09887037037037034
0.09704545454545478
0.09535714285714343
0.09876000000000033
0.09801562499999994
0.09608000000000061
0.09677500000000024
0.0953571428571424
0.09894827586206904
0.09339999999999975
0.09638888888888886
0.09340000000000259
0.08899999999999864
0.09835087719298236
0.09771428571428535
0.08899999999999864
0.09704545454545413
0.09786666666666691
0.08899999999999864
0.09897752808988793
0.09706818181818087
0.09894827586206854
0.09733333333333387
0.09790000000000039
0.09881730769230757
0.0933999999999969
0.09753846153846378
0.09842500000000029
0.09771428571428535
0.09889928057554008
0.09174999999999756
0.09845901639344216
0.09941152263374502
0.0985119047619045
0.0970909090909076
0.0933999999999969
0.09753846153845941
0.09928658536585375
0.09804081632653007
0.09753846153845941
0.09894827586206953
0.0991912568306007
0.09851

In [26]:
# 6.5 add filtered v

for k, df in dfs_dict_with_angles.items():
    sub_list = []
    i = 0
    for ldf in df:
        ldf = ldf.reset_index(drop=True)
        ldf = add_filtered_V(ldf)
        sub_list.append(ldf)
    dfs_dict_with_angles[k] = sub_list

In [34]:
dfs_with_walls.keys()

dict_keys(['tr11', 'tr12', 'tr2', 'tr3', 'tr4', 'tr5', 'tr6', 'tr7', 'tr91', 'tr92'])

In [28]:
# 7. sort by areas of open field
#долго выполняется если рисовать все графики


def sort_walls_area(tr, border_tr, wall_percent=0.1): 
    ylen = border_tr['ymax'] - border_tr['ymin'] 
    xlen = border_tr['xmax'] - border_tr['xmin'] 
    xl = border_tr['xmin'] + xlen * wall_percent
    xr = border_tr['xmax'] - xlen * wall_percent
    yd = border_tr['ymin'] + ylen * wall_percent
    yu = border_tr['ymax'] - ylen * wall_percent

    tr['near_wall'] = 0.
    for i in range(len(tr)):     
        if tr['x'].iloc[i] < xl:
            if tr['y'].iloc[i] > yu:
                tr['near_wall'].iloc[i] = 40
            elif tr['y'].iloc[i] < yd:
                tr['near_wall'].iloc[i] = 30
            else:
                tr['near_wall'].iloc[i] = 4
        elif tr['x'].iloc[i] > xr:
            if tr['y'].iloc[i] > yu:
                tr['near_wall'].iloc[i] = 10
            elif tr['y'].iloc[i] < yd:
                tr['near_wall'].iloc[i] = 20
            else:
                tr['near_wall'].iloc[i] = 2
        elif tr['y'].iloc[i] > yu:
            tr['near_wall'].iloc[i] = 1
        elif tr['y'].iloc[i] < yd:
            tr['near_wall'].iloc[i] = 3
    return tr

def plot_wall_sorted_samples(df, plot_title):
    all_areas = [40, 30, 20, 10, 4, 3, 2, 1, 0]
    for a in all_areas:
        x = df.loc[df['near_wall'] == a]['x']
        y = df.loc[df['near_wall'] == a]['y']
        plt.scatter(x, y, s=1.5, label=str(a))
    plt.gca().set_aspect('equal')
    plt.title(plot_title)
    plt.legend(loc='center')
    plt.show()


dfs_with_walls = {}
bord40 =  {'xmin': -40,'xmax': 40,'ymin': -40,'ymax': 40}   

for k, df in dfs_dict_with_angles.items():
    subtraj_list = []
    for sub_df in df:
        sub_df = sort_walls_area(sub_df, bord40, wall_perc)
        #plot_wall_sorted_samples(sub_df, k)
        subtraj_list.append(sub_df)
    dfs_with_walls[k] = subtraj_list







In [15]:
#dfs_with_walls

In [9]:
# #checked that sampling rate stability in subtrajectories
# for k, df in dfs_with_walls.items():
#     print(k)
#     for ldf in df:
#         s_r = (ldf['time'].max() - ldf['time'].min())/ len(ldf)
#         print(s_r)

tr11
0.03336604017653325
tr12
0.03336518498802236
0.03336519800681878
0.03304854368932035
0.033362012290981846
tr2
0.03336603694213928
tr3
0.13117241379310346
0.11866666666666642
0.13021951219512193
0.12558823529411794
0.12136363636363709
0.12994736842105306
0.13068749999999993
0.12642105263157974
0.12903333333333364
0.12714285714285656
0.1304318181818183
0.11674999999999969
0.12851851851851848
0.11675000000000324
0.11866666666666485
0.1303720930232557
0.1302857142857138
0.11866666666666485
0.1255882352941171
0.1304888888888892
0.11442857142856967
0.1314776119402989
0.12942424242424116
0.13043181818181762
0.1297777777777785
0.12769565217391357
0.13175641025641008
0.11674999999999613
0.1268000000000029
0.13123333333333373
0.1302857142857138
0.132182692307693
0.12233333333333007
0.13056521739130372
0.13273076923076946
0.131349206349206
0.12564705882352747
0.11674999999999613
0.12679999999999722
0.13238211382113835
0.12983783783783712
0.12679999999999722
0.13043181818181893
0.132496350364

In [32]:
#dfs_step_sum.keys()

dict_keys(['tr11', 'tr12', 'tr2', 'tr3', 'tr4'])

In [33]:
# # 7.5 differentiate movements and stops, check percent of movement

# #add sum of steps for the last 0.5 seconds to each row (if applicable)

# dfs_step_sum = {}

# #window_size = round(0.5/0.034)

# for k, df in dfs_with_walls.items():
#     sub_list = []
#     for ldf in df:
#         ldf = ldf.iloc[1:]
#         #first row of ldf does not have step length
#         s_r = (ldf['time'].max() - ldf['time'].min())/ len(ldf)
#         window_size = round(0.5/s_r)
#         if len(ldf) >= window_size:
#             ldf['step_sum_0.5'] = ldf['step_length'].rolling(window_size).sum()
#             ldf['step_sum_0.5']=ldf['step_sum_0.5'].fillna(-100)
#             sub_list.append(ldf)
#         dfs_step_sum[k] = sub_list
        
# #merge dfs in dictionary

# # merg_resampled_dfs_dict = {}

# # for k, df in dfs_step_sum.items():
# #     df = [df0.iloc[1:].reset_index() for df0 in df]
# #     merg_resampled_dfs_dict[k] = pd.concat(df)
# #     merg_resampled_dfs_dict[k] = merg_resampled_dfs_dict[k].reset_index(drop=True)



In [35]:
merg_resampled_dfs_dict = {}

for k, df in dfs_with_walls.items():
    df = [df0.iloc[1:].reset_index() for df0 in df]
    merg_resampled_dfs_dict[k] = pd.concat(df)
    merg_resampled_dfs_dict[k] = merg_resampled_dfs_dict[k].reset_index(drop=True)

In [38]:
merg_resampled_dfs_dict.keys()

dict_keys(['tr11', 'tr12', 'tr2', 'tr3', 'tr4', 'tr5', 'tr6', 'tr7', 'tr91', 'tr92'])

In [40]:
# # Save distributions

#dfs_stop_save = addMovementColumn(merg_resampled_dfs_dict, 2.5)

with open('for_polygons_resampled_0_1.pkl', 'wb') as fp:    
    pickle.dump(merg_resampled_dfs_dict, fp)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(dfs_stop['tr11'][0].loc[800:1100])
